In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

from inst.cadria import wnd

w = wnd()
w.load()

In [ ]:
w.grab()
w.image

In [ ]:
def show_wait(t, multiple_sec=1):
    from ipywidgets import IntProgress
    from IPython.display import display
    p = IntProgress()
    p.max = int(t*multiple_sec+1)
    p.description = 'INIT'
    display(p)
    
    for j in range(p.max):
        time.sleep(1)
        p.value = j + 1
        p.description = str(p.max - j) + ' s'
    p.description = 'DONE'

# 戰爭-籌備

In [ ]:
from inst.cadria import WarProduceMode
def produce(n=5, t=60, 
            itemAvailabes=[True for x in list(range(6))], 
            warmode=WarProduceMode.EASY):
    
    itemAvailabes += [False for x in list(range(6 - len(itemAvailabes)))]
    
    p = IntProgress()
    p.max = t
    p.description = 'INIT'
    display(p)
    
    for _ in range(n):
        # 製作
        w.focus()
        for seq in range(6):
            if itemAvailabes[seq]:
                w.warProduce(warmode)
                time.sleep(1)

        # 等待全部完成
        for j in range(p.max):
            time.sleep(1)
            p.value = j + 1
            p.description = str(p.max - j) + ' s'
        p.description = 'DONE'

        # 收成
        w.focus()
        for seq in range(6):
            if itemAvailabes[seq]:
                w.warHarvest(seq)
                time.sleep(1)


# 戰爭-戰鬥

In [ ]:
def fight(n=1):    
    from inst.cadria import WarFightMode
    
    warmode = WarFightMode.HARD
    
    # 戰爭等待時間(sec)
    t = (10 + 10*warmode) * 60
    
    p = IntProgress()
    p.max = t
    p.description = 'INIT'
    display(p)
    
    # 可用隊伍，依序執行(如果有中斷過則順序要調整)
    teams = ['1', '2']
    
    for i in range(n):
        teamNo = teams[i % len(teams)]
        w.focus()
        w.warFight(teamNo, warmode)
    
        for j in range(p.max):
            time.sleep(1)
            p.value = j + 1
            p.description = str(p.max - j) + ' s'
        p.description = 'DONE'
    
        w.focus()
        w.warFightCompleted()
        time.sleep(1)
    

# 執行

In [ ]:
class warstepType:
    NONE = 0
    PRODUCE = 1
    FIGHT = 2

warstep = warstepType.NONE

import time
import datetime
d1 = datetime.datetime(2019, 3, 11, 9)

while warstep==warstepType.NONE:
    now = datetime.datetime.now()
    df_hour = (now - d1).total_seconds()/60/60 % 168 % 84
    if df_hour<24:
        warstep = warstepType.PRODUCE
    elif df_hour<36:
        show_wait(36-df_hour, 60*60)
    elif df_hour<60:
        warstep = warstepType.FIGHT
    else:
        show_wait(84-df_hour, 60*60)

w.focus()
w.warEnter()
        
if warstep == warstepType.PRODUCE:
    from inst.cadria import WarProduceMode
    produce(t=2*60-2, n=30, warmode=WarProduceMode.EASY)
elif warstep == warstepType.FIGHT:
    fight(20)
else:
    print('not work:', warstep)
    